# labelled sites example

In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
import numpy as np
import operator
from site_analysis import Polyhedron, Atom, Analysis, get_vertex_indices, AtomsTrajectory, SitesTrajectory

Load a `POSCAR` file where every octahedral site is occupied by a Na atom.

In [2]:
all_na_structure = Poscar.from_file('na_sn_all_na_new.POSCAR.vasp').structure
vertex_species = 'S'
centre_species = 'Na'

Create six pymatgen Structures using the `Structure.from_spacegroup()` method, that each only contain the NaX sites, using the coordinates from Ramos _et al._ _Chem. Mater._ 2018.

In [3]:
from pymatgen import Structure, Lattice
lattice = all_na_structure.lattice
na1 = Structure.from_spacegroup(sg=142, lattice=lattice, species=['Na'], coords=[[0.0, 0.0, 0.125]])
na2 = Structure.from_spacegroup(sg=142, lattice=lattice, species=['Na'], coords=[[0.00, 0.25, 0.000]])
na3 = Structure.from_spacegroup(sg=142, lattice=lattice, species=['Na'], coords=[[0.0, 0.0, 0.25]])
na4 = Structure.from_spacegroup(sg=142, lattice=lattice, species=['Na'], coords=[[0.0, 0.0, 0.0]])
na5 = Structure.from_spacegroup(sg=142, lattice=lattice, species=['Na'], coords=[[0.25, 0.5, 0.125]])
na6 = Structure.from_spacegroup(sg=142, lattice=lattice, species=['Na'], coords=[[0.0, 0.25, 0.125]])
na_structures = {'Na1': na1,
                 'Na2': na2,
                 'Na3': na3,
                 'Na4': na4,
                 'Na5': na5,
                 'Na6': na6}

Import the `matching_site_indices()` function from `polyhedral_analysis` (I should probably just add this to the `site-analysis` package).

In [4]:
from polyhedral_analysis.polyhedra_recipe import matching_site_indices

In [5]:
print(matching_site_indices.__doc__)


    Returns a subset of site indices from structure (as a list) where each site is the closest to one 
    site in the reference structure.
    
    Args:
        structure (Structure): The structure being analysed.
        reference_structure (Structure): A Structure object containing a set of reference sites.
        species (:obj:`list[str]`, optional): A list of species labels. If this is set, only matching
            sites will be included in the returned set.
        
    Returns:
        (list[int])
    


In [6]:
matching_site_indices(all_na_structure, na_structures['Na1'])

[9, 92, 68, 91, 66, 47, 12, 36, 99, 64, 14, 29, 100, 65, 13, 94]

Use this to find the Na sites closest to a Na1, Na2, Na3 etc site, and store the site index and "Na1" etc. label in a dictionary, using the index as keys. Then sort this dictionary and generate an ordered list of "Na1" etc. labels for every Na site.

In [7]:
labels = {}
for l, structure in na_structures.items():
    indices = matching_site_indices(all_na_structure, structure)
    for i in indices:
        labels[i] = l
sorted_labels = [ labels[i] for i in sorted(labels) ]
print( sorted_labels )

['Na2', 'Na2', 'Na2', 'Na4', 'Na2', 'Na3', 'Na2', 'Na2', 'Na2', 'Na1', 'Na2', 'Na3', 'Na1', 'Na1', 'Na1', 'Na6', 'Na6', 'Na4', 'Na1', 'Na4', 'Na3', 'Na2', 'Na2', 'Na2', 'Na2', 'Na1', 'Na5', 'Na6', 'Na2', 'Na2', 'Na2', 'Na2', 'Na5', 'Na5', 'Na5', 'Na5', 'Na1', 'Na3', 'Na4', 'Na6', 'Na6', 'Na5', 'Na6', 'Na6', 'Na1', 'Na1', 'Na1', 'Na4', 'Na1', 'Na3', 'Na3', 'Na4', 'Na5', 'Na6', 'Na6', 'Na5', 'Na6', 'Na6', 'Na5', 'Na1', 'Na1', 'Na3', 'Na1', 'Na5', 'Na5', 'Na4', 'Na3', 'Na1', 'Na1', 'Na4', 'Na5', 'Na6', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na2', 'Na6', 'Na6', 'Na5', 'Na5', 'Na5', 'Na6', 'Na6', 'Na5']


We then use the `get_vertex_indices()` function to find the six closest S to each Na (within a cutoff of 4.3 Å).  
This returns a nested list, where each sublist contains the S indices for a single polyedron.  
Note: this index counts from 1, and ignores other species in the structure (so is not affected by species order).

In [8]:
# find atom indices (within species) for all polyhedra vertex atoms
vertex_indices = get_vertex_indices(all_na_structure, centre_species=centre_species, 
                                    vertex_species=vertex_species, cutoff=4.3)
print(vertex_indices[:4])

[[27 29 59 61 83 85]
 [19 21 51 53 91 93]
 [ 9 15 41 47 65 71]
 [ 7 23 33 49 73 90]]


We can now use these vertex ids to define our `Polyhedron` objects.   
We now also pass in the appropriate label to each polyhedron.

In [9]:
structure = Poscar.from_file('POSCAR').structure
# create Polyhedron objects
polyhedra = [Polyhedron(vertex_species=vertex_species, vertex_indices=vi, label=label) 
             for vi, label in zip(vertex_indices, sorted_labels)]

In [10]:
polyhedra[0].as_dict()

{'index': 1,
 'vertex_species': 'S',
 'vertex_indices': array([27, 29, 59, 61, 83, 85]),
 'vertex_coords': None,
 'contains_atoms': [],
 'label': 'Na2'}

In [11]:
polyhedra[34].as_dict()

{'index': 35,
 'vertex_species': 'S',
 'vertex_indices': array([ 3,  5, 35, 37, 76, 78]),
 'vertex_coords': None,
 'contains_atoms': [],
 'label': 'Na5'}

To analyse the site occupation for a particular `pymatgen` `Structure`:

In [ ]:
analysis.analyse_structure(structure)

The list of sites occupied by each atom can now be accessed using `analysis.atom_sites`

In [ ]:
np.array(analysis.atom_sites)

The occupations of each site are stored as a list of lists, as each site can have zero, one, or multiple atoms occupying it.

In [ ]:
analysis.site_occupations[:4]

There are two ways to think about a site-projected trajectory:  
1. From an atom-centric perspective. Each atom visits a series of sites, and occupies one site each timestep.
2. From a site-centric perspective. Each site is visited by a series of atoms, and has zero, one, or more atoms occupying it at each timestep.

These two trajectory types are handled with the `AtomTrajectory` and `SiteTrajectory` classes:

In [ ]:
at = AtomsTrajectory(atoms)
st = SitesTrajectory(polyhedra)

The `AtomTrajectory` and `SiteTrajectory` classes provide convenient wrappers for storing sequences of site-occupation data. Both classes have `append_timestep()` methods, e.g. to add analysis data at $t=1$:

In [ ]:
at.append_timestep(analysis.atom_sites, t=1)
st.append_timestep(analysis.site_occupations, t=1)

In [ ]:
print(at.timesteps)
print(at.data)

In [ ]:
print(st.data[0][:4])
print(st.timesteps)

An `Analysis` object also has an `append_timestep()` method, that updates the `atoms_trajectory`, `sites_trajectory`, and `timesteps` attributes:

In [ ]:
analysis.reset()
analysis.append_timestep(structure,t=1)
print(analysis.timesteps)
print(analysis.atoms_trajectory.data[0])

Example of processing a simulation trajectory using the `XDATCAR` file:

In [ ]:
%%timeit
analysis.reset()

xdatcar = Xdatcar('XDATCAR')

for timestep, s in enumerate(xdatcar.structures):
    analysis.append_timestep(s, t=timestep)

Checking which sites has Na(4) visited:  
(note use of `analysis.at` as shorthand for `analysis.atoms_trajectory`)

In [ ]:
analysis.at.by_atom_index(4) # convert to a numpy array to then use numpy array slicing to extract a single atom trajectory.

Na(4) starts in site 15, and moves to site 73 at timestep 5.  
The same information can be seen by querying the site occupation data for sites 15 and 73:  
(note use of `analysis.st` as shorthand for `analysis.sites_trajectory`)

In [ ]:
print(analysis.st.by_site_index(15))
print(analysis.st.by_site_index(73))

In [ ]:
d = atoms[0].to()
d

In [ ]:
atoms[0].to('atom.json')

In [ ]:
Atom.from_file('atom.json').frac_coords

In [ ]:
d = polyhedra[0].as_dict()
d

In [ ]:
p = Polyhedron.from_dict(d)

In [ ]:
p.vertex_coords

In [ ]:
d = analysis.at.to_dict()

In [ ]:
AtomsTrajectory.from_dict(d).atom_lookup

In [ ]:
d = analysis.st.to_dict()
d

In [ ]:
SitesTrajectory.from_dict(d).site_lookup